In [2]:
import os
import cv2
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, applications, Input
from scipy.stats import rankdata
from scipy.ndimage import gaussian_filter1d

# ==========================================
# 1. CONFIGURATION
# ==========================================
TRAIN_DIR = '/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/training_videos'
TEST_DIR = '/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/testing_videos'

IMG_SIZE = 128
SEQ_LEN = 8                # Context Window
FEATURE_DIM = 2048         # ResNet Output
INPUT_DIM = SEQ_LEN * FEATURE_DIM  # 16,384 (Flattened Input)
OUTPUT_DIM = FEATURE_DIM           # 2,048 (Target is ONE frame)

BATCH_SIZE = 120
EPOCHS = 150

# ==========================================
# 2. FEATURE EXTRACTOR (ResNet101)
# ==========================================
def build_feature_extractor():
    base_model = applications.ResNet101(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D()
    ])
    return model

print("⏳ Loading ResNet-101...")
feature_model = build_feature_extractor()
print("✅ ResNet-101 Loaded.")

# ==========================================
# 3. YOUR EXACT ARCHITECTURE (Adapted I/O)
# ==========================================
def build_middle_frame_predictor():
    # 1. INPUT: 8 Frames (16,384)
    inputs = layers.Input(shape=(INPUT_DIM,))
    
    # --- YOUR EXACT LAYERS (Sigmoid + BN) ---
    x = layers.GaussianNoise(0.1)(inputs)
    
    # Layer 1 (Wide Input)
    x = layers.Dense(1536)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('sigmoid')(x)
    
    # Layer 2 (Encoder)
    x = layers.Dense(50)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('sigmoid')(x)
    
    # Layer 3 (Bottleneck)
    code = layers.Dense(30)(x)
    code = layers.BatchNormalization()(code)
    code = layers.Activation('sigmoid')(code)
    
    # Layer 4 (Decoder)
    x = layers.Dense(50)(code)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('sigmoid')(x)
    
    # Layer 5 (Wide Output)
    x = layers.Dense(1536)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('sigmoid')(x)
    
    # 2. OUTPUT: Predict ONLY Middle Frame (2048)
    # Using Linear activation for reconstruction is standard
    outputs = layers.Dense(OUTPUT_DIM, activation='linear')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

model = build_middle_frame_predictor()
print(f"✅ Model Built: Input({INPUT_DIM}) -> [Your Layers] -> Output({OUTPUT_DIM})")

# ==========================================
# 4. UTILS: SORTING & PROCESSING
# ==========================================
def get_sorted_files(directory):
    """Ensures frames are processed in temporal order: 1, 2, 3..."""
    all_files = []
    video_folders = sorted(os.listdir(directory))
    
    for vid in video_folders:
        vid_path = os.path.join(directory, vid)
        if not os.path.isdir(vid_path): continue
        
        frames = sorted(os.listdir(vid_path))
        for f in frames:
            if f.lower().endswith(('.jpg', '.png')):
                all_files.append(os.path.join(vid_path, f))
    return all_files

def extract_features(files, model):
    """Extracts 2048-dim vectors."""
    features = []
    batch_imgs = []
    
    print(f"   Extracting features for {len(files)} frames...")
    
    for i, p in enumerate(files):
        img = cv2.imread(p)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype('float32') / 255.0 
        batch_imgs.append(img)
        
        if len(batch_imgs) == 64 or i == len(files) - 1:
            batch_arr = np.array(batch_imgs)
            batch_arr = applications.resnet.preprocess_input(batch_arr * 255.0)
            
            preds = model.predict(batch_arr, verbose=0)
            features.append(preds)
            batch_imgs = []
            
    return np.vstack(features)

# ==========================================
# 5. DATA PREPARATION (Windowing)
# ==========================================
def create_dataset(raw_features, seq_len=8, is_test=False):
    """
    Creates (Input, Target) pairs.
    Input:  Flattened Window (16384)
    Target: Middle Frame (2048)
    """
    X = []
    Y = []
    
    # Handle Border Padding for Test (to keep exact frame count)
    if is_test:
        pad_start = np.tile(raw_features[0], (seq_len // 2, 1))
        pad_end = np.tile(raw_features[-1], (seq_len // 2, 1))
        # Ensure we have enough padding to center every original frame
        # If seq_len is 8, we target the 4th frame (index 3 or 4).
        features_to_process = np.vstack([pad_start, raw_features, pad_end])
        
        # We slide over the padded array, but we want exactly N outputs
        limit = len(raw_features)
    else:
        features_to_process = raw_features
        limit = len(raw_features) - seq_len + 1

    for i in range(len(features_to_process) - seq_len + 1):
        # 1. Input: The Window
        window = features_to_process[i : i+seq_len].flatten()
        
        # 2. Target: The Middle Frame
        # If window is [0,1,2,3,4,5,6,7], Middle is 4
        mid_idx = i + (seq_len // 2)
        if mid_idx < len(features_to_process):
            target = features_to_process[mid_idx]
            
            X.append(window)
            Y.append(target)
            
        if is_test and len(X) == len(raw_features):
            break
            
    return np.array(X), np.array(Y)

# ==========================================
# 6. TRAINING PHASE
# ==========================================
print("\n--- 🚀 PHASE 1: TRAINING ---")
train_files = get_sorted_files(TRAIN_DIR)
train_raw = extract_features(train_files, feature_model)

# Create Training Data (Drop borders is fine)
train_X, train_Y = create_dataset(train_raw, SEQ_LEN, is_test=False)

print(f"Training on {train_X.shape[0]} samples...")
model.fit(train_X, train_Y, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

# ==========================================
# 7. INFERENCE PHASE
# ==========================================
print("\n--- 🚀 PHASE 2: INFERENCE ---")
test_files = get_sorted_files(TEST_DIR)
test_raw = extract_features(test_files, feature_model)

# Create Test Data (With Padding -> 1 Prediction per Frame)
test_X, test_Y_Actual = create_dataset(test_raw, SEQ_LEN, is_test=True)

print(f"Predicting {test_X.shape[0]} frames...")
test_Y_Pred = model.predict(test_X, batch_size=64, verbose=1)

# Calculate MSE (Prediction vs Actual Middle Frame)
mse = np.mean(np.square(test_Y_Actual - test_Y_Pred), axis=1)

# ==========================================
# 8. SAVE SUBMISSION
# ==========================================
def get_clean_id(p):
    parts = p.split(os.sep)
    try:
        f_num = int(re.findall(r'\d+', parts[-2])[-1])
        i_num = int(re.findall(r'\d+', parts[-1])[-1])
        return f"{f_num}_{i_num}"
    except:
        return "0_0"

ids = [get_clean_id(p) for p in test_files]

# Align lengths (Safety check)
limit = min(len(ids), len(mse))
df = pd.DataFrame({'Id': ids[:limit], 'MSE': mse[:limit]})

# PSNR & Normalization
epsilon = 1e-10
df['Predicted'] = rankdata(df['MSE']) / len(df)
df['PSNR'] = 10 * np.log10(1.0 / (df['Predicted'] + epsilon))

p_min, p_max = df['PSNR'].min(), df['PSNR'].max()
df['Anomaly_Score'] = 1 - ((df['PSNR'] - p_min) / (p_max - p_min + epsilon))

# NO SMOOTHING (Experiment)
df['Smoothed'] = df['Anomaly_Score'] 

# Final Save
final_csv = df[['Id', 'Smoothed']].rename(columns={'Smoothed': 'Predicted'})
final_csv.to_csv('submission_middle_frame_prediction.csv', index=False)

print("\n✅ DONE! 'submission_middle_frame_prediction.csv' saved.")
print("   - Strategy: Temporal Prediction (Input 8 -> Output 1)")
print("   - Architecture: Preserved (1536->50->30...)")


⏳ Loading ResNet-101...
✅ ResNet-101 Loaded.
✅ Model Built: Input(16384) -> [Your Layers] -> Output(2048)

--- 🚀 PHASE 1: TRAINING ---
   Extracting features for 9204 frames...
Training on 9197 samples...
Epoch 1/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - loss: 0.5494
Epoch 2/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1119
Epoch 3/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0904
Epoch 4/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0814
Epoch 5/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0765
Epoch 6/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0733
Epoch 7/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0679
Epoch 8/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0670
Epoch 9/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0637
Epoch 10/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0611
Epoch 11/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0610
Epoch 12/150
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - l